In [ ]:
#!pip install -U langchain-openai

In [46]:
import pandas as pd
import numpy as np
import time
import pathlib
import os
import re

from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
#from langchain_core.prompts import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from typing import List
from dotenv import load_dotenv

# Load environment variables

In [23]:
path_env = pathlib.Path('/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env')
print(f"Ruta al archivo .env: {path_env}")

load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key

Ruta al archivo .env: /export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env


## Create VectorDB with parquet docs

In [12]:
path = '/export/usuarios_ml4ds/cggamella/NP-Company-Process/data/DESCARGAS_ENTREGABLES/outsiders.parquet'
df_out = pd.read_parquet(path)

def unify_colname(col):
    return ".".join([el for el in col if el])

df_out.columns = [unify_colname(col) for col in df_out.columns]

index_names = df_out.index.names
#Se resetea el índice sobre el propio dataframe
df_out.reset_index(inplace=True)
#Crear identifier con los index_names separados con '/', así generamos un id único para cada fila
df_out["identifier"] = df_out[index_names].astype(str).agg("/".join, axis=1)
#Filtrado para quedarme con esas 2 cols
df_out = df_out[['identifier', 'title']]

In [13]:
# Crear el text splitter, [chunk_size: #caracteres de cada chunk];
#[chunk_overlap: #caracteres solapan entre chunks para no perder info.]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Crear una lista para almacenar los documentos
documents = []
# Procesar cada fila como un documento separado
for idx, row in df_out.iterrows():
    example_document = row['title']
    #print("el example doc es:",example_document)
    doc = Document(page_content=example_document, metadata={"url": "local", "source": "initial", "identifier": row['identifier']})
    #print("El doc es:",doc)
    # Dividir el documento en fragmentos
    chunks = text_splitter.split_text(doc.page_content)
    #print("Los chunks son:",chunks)
    for chunk in chunks:
        chunk_doc = Document(page_content=chunk, metadata=doc.metadata)
        #print("EL chunk_doc es:\n",chunk_doc)
        documents.append(chunk_doc)

# Retriever personalizado

In [15]:
# Embed and store the texts
path_to_index = '/export/usuarios_ml4ds/cggamella/NP-Search-Tool/aux_scripts/RAG'
# Supplying a persist_directory will store the embeddings on disk
persist_directory = (pathlib.Path(path_to_index) / 'db').as_posix()

In [17]:
#Cargar la bbdd vectorial de Chroma(ya creada) que contiene docs, embeddings y metadatos 'identifier'
vectorstore = Chroma(persist_directory=persist_directory)

In [31]:
class AcronymAwareRetriever(BaseRetriever):
    """
    Un Retriever personalizado que busca documentos que contienen un acrónimo específico.
    """
    documents: List[Document]
    k: int

    def _get_relevant_documents(
        self, acronym: str, *, run_manager: CallbackManagerForRetrieverRun = None
    ) -> List[Document]:
        """
        Recupera documentos que contienen el acrónimo proporcionado.
        """
        matching_documents = []
        pattern = re.compile(r'\b{}\b'.format(re.escape(acronym)), re.IGNORECASE)

        for document in self.documents:
            if len(matching_documents) >= self.k:
                break
            if pattern.search(document.page_content):
                matching_documents.append(document)
        
        return matching_documents

In [21]:
# Crear el retriever con una lista de documentos y el número máximo de resultados a recuperar
acronym_retriever = AcronymAwareRetriever(documents=documents, k=2)
result_docs = acronym_retriever.invoke("ceip")

# Print the results
for doc in result_docs:
    print(doc.page_content, "\n")

OBRA COMPLEMENTARIA para la Construcción de un CEIP en Arroyo de la Encomienda (Valladolid) 

Subsanación de deficiencias de revisión del R.E.B.T. en CEIP Antonio Fontán en Avda. Santuario de Valverde, 71 c/v Monasterio de Arlanza. Distrito Fuencarral-El Pardo 



# Crear bbdd y guardar embeddings

In [ ]:
'''
start = time.time()
# Define embeddings
#embedding = OpenAIEmbeddings()

# Almacenar los fragmentos en una base de datos vectorial usando Chroma
# Se extrae el contenido (page_content).El contenido se pasa a OpenAIEmbeddings
# para obtener embeddings. El vector resultante se almacena en la base de datos junto con los metadatos(índices).
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_directory 
)
# Tiempo total de ejecución, tarda 51 mins en 300k docs
end = time.time()
print(f"Total time is {end - start} seconds")
'''

In [48]:
# Tu prompt_template
prompt_template_text = """
Given the following {summaries}. These documents are related or have the context to understand the following task:

Given the word "{acronym}", understand that an acronym word is a type of abbreviation formed by taking the initial
letters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word. And after reading the previous documents I sent you,
please provide the full expression of the acronym in the following dictionary format: 'acronym:expanded_word',
where spaces between words are replaced by underscores '_'. If the expanded word is too long or if there is any other issue, do NOT provide an answer.
--------------------
If you find any issue finding the correct expression, your answer should have the following format: ''. Otherwise, your answer should be 'acronym:expanded_word'.

For example, for the following acronyms, "aeat" stands for "agencia_estatal_de_administración_tributaria", "csic" stands for "consejo_superior_de_investigaciones_científicas", "ceip" stands for "colegio_educación_infantil_y_primaria", and "avda" stands for "avenida".
Additionally, abbreviations can be formed by taking the initial letters of a phrase (like acronyms), by using only some letters of a word (like "Dr." for "Doctor"), or by shortening a word (like "apt." for "apartmento"). 
"""

prompt_template = PromptTemplate.from_template(template=prompt_template_text)

In [50]:
# Set up the turbo LLM
llm = ChatOpenAI(
    temperature=0.2,
    model_name='gpt-4o'
)

# Crear el retriever personalizado con una lista de documentos
acronym_retriever = AcronymAwareRetriever(documents=documents, k=3)

# Crear la chain
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    retriever=acronym_retriever,
    chain_type="stuff",
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt_template
    }
)

In [51]:
def generate_acronym_expansions(chain, acronyms):
    results = {}
    for acronym in acronyms:
        # Recuperar documentos relevantes utilizando el retriever
        retrieved_docs = chain.retriever.get_relevant_documents(acronym)
        print("Los RETRIEVED DOCS SON:\n", retrieved_docs)
        # Crear el resumen de documentos
        summaries = "\n".join([doc.page_content for doc in retrieved_docs])
        # Formar el prompt utilizando el template
        prompt = prompt_template.format(acronym=acronym, summaries=summaries)
        # Ejecutar la cadena con el prompt usando invoke
        response = chain.invoke({"question": prompt, "acronym": acronym, "summaries": summaries})
        print(f"Respuesta del modelo para '{acronym}':\n{response}\n")
        # Obtener la respuesta del modelo de lenguaje
        answer = response['answer']
        print(f"LA ANSWER ES:\n",answer)
        # Parsear la respuesta y agregarla a los resultados
        results[acronym] = answer.strip()
    return results

In [52]:
# Lista de acrónimos a buscar
acronyms = ['ghz','ecc','vga','s.l', 's.a.', 'bop', 'pcap', 'ceip', 'jjmm', 'smp', 'avda', 'ffcc', 'itv']

# Generar las expansiones de los acrónimos
acronym_expansions = generate_acronym_expansions(chain, acronyms)

# Imprimir los resultados
for acronym, expansion in acronym_expansions.items():
    print(f"{acronym}: {expansion}")

/home/cggamella/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Los RETRIEVED DOCS SON:
 [Document(page_content="El present contracte té per objecte el subministrament de 34 ordinadors personals, compostos per monitor més torre, per al servei de biblioteques, ubicat al carrer Bernat Boades 10, de Girona. Aquests tindran, com a mínim, les característiques següents:\n\nChipset: Intel® Q270 \nProcessador Intel® Core¿ i5 7ª Gen 7500 3.4 Ghz (3.8 Ghz) (Quad Core)\nPlaca gràfica HD 530\nSistema operatiu Microsoft Windows  10 Professional 64 bits\nMemòria 4 GB (Instal¿lats)DDR4 SDRAM ¿ no ECC\nDisc dur de 500 Gb SATA 7200 rpm\nUnitat òptica: Gravadora DVD\nTeclat i ratolí\nTarja xarxa Ethernet 10/100/1000 integrada \nConnectors frontals  2 USB 3.0, 2 USB 2.0 (1 carga ràpida), connectors de micròfon i auriculars \nFormat minitorre o menor.\nPorts:\n1 VGA\n2 DisplayPort\n1 RJ-45\n1 HDMI\n1 línia d'entrada d'audio\n1 línia de sortida d'audio\n \nSeguretat: Activació/desactivació USB\nControl d'arrencada mitjançant suport extraïbles\n\n\nMonitor", metadata={'

Los RETRIEVED DOCS SON:
 [Document(page_content="El present contracte té per objecte el subministrament de 34 ordinadors personals, compostos per monitor més torre, per al servei de biblioteques, ubicat al carrer Bernat Boades 10, de Girona. Aquests tindran, com a mínim, les característiques següents:\n\nChipset: Intel® Q270 \nProcessador Intel® Core¿ i5 7ª Gen 7500 3.4 Ghz (3.8 Ghz) (Quad Core)\nPlaca gràfica HD 530\nSistema operatiu Microsoft Windows  10 Professional 64 bits\nMemòria 4 GB (Instal¿lats)DDR4 SDRAM ¿ no ECC\nDisc dur de 500 Gb SATA 7200 rpm\nUnitat òptica: Gravadora DVD\nTeclat i ratolí\nTarja xarxa Ethernet 10/100/1000 integrada \nConnectors frontals  2 USB 3.0, 2 USB 2.0 (1 carga ràpida), connectors de micròfon i auriculars \nFormat minitorre o menor.\nPorts:\n1 VGA\n2 DisplayPort\n1 RJ-45\n1 HDMI\n1 línia d'entrada d'audio\n1 línia de sortida d'audio\n \nSeguretat: Activació/desactivació USB\nControl d'arrencada mitjançant suport extraïbles\n\n\nMonitor", metadata={'



> Entering new RetrievalQAWithSourcesChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following . These documents are related or have the context to understand the following task:

Given the word "vga", understand that an acronym word is a type of abbreviation formed by taking the initial
letters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word. And after reading the previous documents I sent you,
please provide the full expression of the acronym in the following dictionary format: 'acronym:expanded_word',
where spaces between words are replaced by underscores '_'. If the expanded word is too long or if there is any other issue, do NOT provide an answer.
--------------------
If you find any issue finding the correct expression, your answer should have the following format: ''. Otherwise, your answer should be 'acronym:expanded_word'.

For


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 's.l':
{'question': '\nGiven the following Licitación para la contratación de suministro de plástico para INDESA 2010, S.L por procedimiento abierto y tramitación ordinaria.\nServei de recollida, transport i lliurament de documentació i / o paqueteria a través de la modalitat de ruta fixa dins de tot l\'àmbit territorial de la societat de Prestacions d\'Assistència Mèdica S.L., així com de les modalitats de missatgeria no programada a la ruta fixa.\nL\'objecte del present contracte és el servei de recollida i transport de substàncies biològiques dins de tot l\'àmbit territorial de la societat de Prestacions d\'Assistència Mèdica, S.L.. These documents are related or have the context to understand the following task:\n\nGiven the word "s.l", understand that an acronym word is a type of abbreviation formed by taking the initial\nletters or parts of words from a phrase or term and combining them to form a n



> Entering new RetrievalQAWithSourcesChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following . These documents are related or have the context to understand the following task:

Given the word "bop", understand that an acronym word is a type of abbreviation formed by taking the initial
letters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word. And after reading the previous documents I sent you,
please provide the full expression of the acronym in the following dictionary format: 'acronym:expanded_word',
where spaces between words are replaced by underscores '_'. If the expanded word is too long or if there is any other issue, do NOT provide an answer.
--------------------
If you find any issue finding the correct expression, your answer should have the following format: ''. Otherwise, your answer should be 'acronym:expanded_word'.

For


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'pcap':
{'question': '\nGiven the following L\'objecte d\'aquest contracte és la realització d\'una enquesta representativa amb els paràmetres bàsics següents:\nRealització d\'una enquesta personal a una mostra de 1.200 i amb una durada màxima de 30 minuts en les condicions que es determinen en el document d\'execució del contracte, en el PCAP i en el PPT de l\'Acord marc.\nAssegurança del parc mòbil de vehicles de l\'Ajuntament de Palafolls. El parc mòbil està integrat pels vehicles relacionats en l\'annex 1 del PCAP i pels vehicles que s\'hi integrin posteriorment durant la vigència del contracte.\nL\'objecte de la licitació és la Contractació de l\'ús, llicenciament, implementació, manteniment, assistència tècnica, suport i serveis associats d\'una aplicació informàtica per a la gestió integral dels recursos humans de l\'Ajuntament de Lleida.\nPer garantir una implantació eficient i sostenible així co

Los RETRIEVED DOCS SON:
 [Document(page_content="Obres compreses en el projecte executiu de la piscina descoberta de 50 metres per construir en l'àmbit de l'anella mediterrània dels JJMM 2018, aprovat definitivament pel Consell Plenari de data 28 de novembre de 2016", metadata={'url': 'local', 'source': 'initial', 'identifier': 'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_200111.atom/388'})]


> Entering new RetrievalQAWithSourcesChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following . These documents are related or have the context to understand the following task:

Given the word "jjmm", understand that an acronym word is a type of abbreviation formed by taking the initial
letters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word. And after reading the previous documents I sent you,
please provide th

Los RETRIEVED DOCS SON:
 [Document(page_content='Renovación de red de abastecimiento en avda. de Las Farolas y otras calles en Ceutí (Murcia)', metadata={'url': 'local', 'source': 'initial', 'identifier': 'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/119'}), Document(page_content='Subsanación de deficiencias de revisión del R.E.B.T. en CEIP Antonio Fontán en Avda. Santuario de Valverde, 71 c/v Monasterio de Arlanza. Distrito Fuencarral-El Pardo', metadata={'url': 'local', 'source': 'initial', 'identifier': 'PlataformasAgregadasSinMenores_2018.zip/PlataformasAgregadasSinMenores_20180217_180137.atom/79'}), Document(page_content='Contrato derivado 5D-PI de las obras de adecuación de las instalaciones generales en el Polideportivo San Blas, sito en la Avda. de Hellín, 59 (Arcos de Jalón, 59) del Distrito de San Blas-Canillejas dentro del Acuerdo marco de las obras de reforma, reparación y conservación del conjunto de edificios cuya competencia


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'ffcc':
{'question': '\nGiven the following Urbanización parcial de la acera de los números impares de la avda. cataluña entre ffcc a huesca y la c/ rio ara\nObres compreses a la Memòria valorada dels treballs de millora de l\'accessibilitat a l\'Av. Ramon d\'Olzina, al pas pel pont del FFCC.\nContracte de serveis per a l\'assistència tècnica de coordinació de seguretat i salut de les obres de la via ciclista de la BV-5301. Santa Maria de Palautordera - Estació FFCC. Clau: XB-13313. These documents are related or have the context to understand the following task:\n\nGiven the word "ffcc", understand that an acronym word is a type of abbreviation formed by taking the initial\nletters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word. And after reading the previous documents I sent you,\nplease provide the full expression of the acronym in the

In [53]:
acronym_expansions

{'ghz': "'acronym:gigahertz'",
 'ecc': "'ecc:'",
 'vga': "'acronym:video_graphics_array'",
 's.l': "''",
 's.a.': "'s.a.:sociedad_anónima'",
 'bop': "'acronym:expanded_word'",
 'pcap': '```\npcap:packet_capture\n```',
 'ceip': "'ceip:colegio_educación_infantil_y_primaria'",
 'jjmm': "''",
 'smp': '```\nsmp:simulación_de_montecarlo\n```',
 'avda': "'avda:avenida'",
 'ffcc': "'acronym:fuerzas_y_cuerpos_de_seguridad'",
 'itv': "'acronym:independent_television'"}

# NP-Search-Tool sacar acrónimos con RAG

In [ ]:
#!pip install chromadb
#!pip install -U langchain-openai

In [54]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from sentence_transformers import SentenceTransformer
import re

In [55]:
path = '/export/usuarios_ml4ds/cggamella/NP-Company-Process/data/DESCARGAS_ENTREGABLES/outsiders.parquet'
df_out = pd.read_parquet(path)

In [56]:
def unify_colname(col):
    return ".".join([el for el in col if el])

In [57]:
df_out.columns = [unify_colname(col) for col in df_out.columns]

In [58]:
#Para coger ['zip', 'file name', 'entry']
index_names = df_out.index.names
#Se resetea el índice sobre el propio dataframe
df_out.reset_index(inplace=True)
#Pone como identifier los index names separados con '/', así generamos un id único para cada fila
df_out["identifier"] = df_out[index_names].astype(str).agg("/".join, axis=1)

In [59]:
df_filtered = df_out.loc[:100, ['identifier', 'title']]

In [60]:
# Calcular la longitud media de la columna 'title'
df_filtered['title_length'] = df_filtered['title'].apply(len)
average_length = df_filtered['title_length'].mean()
average_length

151.6039603960396

In [61]:
# Inicializar el modelo de embeddings
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device='cuda')
# Generar embeddings para cada título en el dataframe
embeddings = model.encode(df_out['title'].tolist())

In [62]:
def retrieve_context_similarity(acronym, embeddings, df_out):
    acronym_embedding = model.encode([acronym])
    #print("La query embedding es:\n", query_embedding)
    similarities = cosine_similarity(acronym_embedding, embeddings)[0]
    print("Las cosine similarities son:\n", similarities)
    most_similar_indices = np.argsort(similarities)[-10:][::-1]
    print("Los indices mas similares son:\n", most_similar_indices)
    results = [df_out['title'].iloc[i] for i in most_similar_indices]
    print("Los resultados del retrieve son:\n", results)
    return results

In [63]:
def retrieve_context_contain_acronym(acronym, df_out):
    # Crear el patrón de expresión regular para coincidir con la palabra exacta
    pattern = re.compile(r'\b' + re.escape(acronym) + r'\b', re.IGNORECASE)
    # Filtrar los títulos que contienen el acrónimo exacto
    results = df_out[df_out['title'].str.contains(pattern, na=False)]
    # Obtener los títulos y limitar a los primeros 5 documentos
    titles = results['title'].head(5).tolist()
    # Limitar cada título a los primeros 70 caracteres
    limited_titles = [title[:70] for title in titles]
    return limited_titles

In [69]:
def retrieve_context_contain_acronym_improved(acronym, df_out):
    # Crear el patrón de expresión regular para coincidir con la palabra exacta
    pattern = re.compile(r'\b' + re.escape(acronym) + r'\b', re.IGNORECASE)
    contexts = []
    
    # Iterar sobre cada título en el DataFrame
    for title in df_out['title']:
        if pd.isna(title):
            continue
        # Buscar todas las coincidencias del patrón en el título
        for match in re.finditer(pattern, title):
            # Asegurarse de no ir más allá del comienzo del texto
            start = max(match.start() - 70, 0)  
            end = match.end() + 70  
            # Extraer el contexto alrededor del acrónimo
            context = title[start:end]
            contexts.append(context)
            # Limitar a los primeros 5 documentos
            if len(contexts) >= 5:  
                return contexts
    
    return contexts

In [70]:
!pip install -U langchain-openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable


In [71]:
llm = OpenAI()
             
def generate_equivalent_expression(acronym):
    contexts = retrieve_context_contain_acronym_improved(acronym, df_out)    
    context_texts = " ".join(contexts)
    prompt = f"Given the acronym {acronym}, and the following context: {context_texts}, provide the large expression of the acronym. Please provide me the result in a dictionary format 'acronym:equivalence'"
    print("EL PROMPTING ES:", prompt)
    response = llm.generate([prompt])
    print("La respuesta es:", response)
    return response.generations[0][0].text 

In [72]:
acronyms = ['ghz','ecc','vga','s.l', 's.a.', 'bop', 'pcap', 'ceip', 'jjmm', 'smp', 'avda', 'ffcc', 'itv']  

In [73]:
results = {}
for acronym in acronyms:
    expression = generate_equivalent_expression(acronym)
    results[acronym] = expression

EL PROMPTING ES: Given the acronym ghz, and the following context: s:

Chipset: Intel® Q270 
Processador Intel® Core¿ i5 7ª Gen 7500 3.4 Ghz (3.8 Ghz) (Quad Core)
Placa gràfica HD 530
Sistema operatiu Microsoft et: Intel® Q270 
Processador Intel® Core¿ i5 7ª Gen 7500 3.4 Ghz (3.8 Ghz) (Quad Core)
Placa gràfica HD 530
Sistema operatiu Microsoft Windows   d'un perfilador vertical Doppler (radar meteorològic) de banda K (24 GHz) per a l'estudi de perfils de precipitació atmosfèrica amb elevada re connexió mitjançant una xarxa de radiofreqüència en banda lliure de 5 GHz en tecnologia basada en 802.11a, 802.11n, 802.11ac o bé 802.16 per la Subministrament d'un generador vectorial de 20 Ghz amb modulacio I/Q de 2 GhZ d'ample de banda i 2G mostres de memòria p, provide the large expression of the acronym. Please provide me the result in a dictionary format 'acronym:equivalence'
La respuesta es: generations=[[Generation(text='\n\nGHz: gigahertz', generation_info={'finish_reason': 'stop', 'logp

La respuesta es: generations=[[Generation(text='\n\nCEIP: Centro de Educación Infantil y Primaria (Center for Infant and Primary Education)', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'total_tokens': 241, 'completion_tokens': 21, 'prompt_tokens': 220}, 'model_name': 'gpt-3.5-turbo-instruct'} run=[RunInfo(run_id=UUID('fffc43b3-4187-4526-baf6-cf692be4e09b'))]
EL PROMPTING ES: Given the acronym jjmm, and the following context: a de 50 metres per construir en l'àmbit de l'anella mediterrània dels JJMM 2018, aprovat definitivament pel Consell Plenari de data 28 de novemb, provide the large expression of the acronym. Please provide me the result in a dictionary format 'acronym:equivalence'
La respuesta es: generations=[[Generation(text='\n\nJJMM: Jocs del Mediterrani del 2018', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'total_tokens': 97, 'completion_tokens': 13, 'prompt_tokens': 84}, 'model_nam

In [74]:
results

{'ghz': '\n\nGHz: gigahertz',
 'ecc': "\n\n'ECC: Error Checking and Correction'",
 'vga': '\n\nvga: Video Graphics Array',
 's.l': '\n\nS.L: Sociedad Limitada (Spanish for Limited Company)',
 's.a.': '\n\ns.a.: Sociedad Anónima (Spanish for "public limited company")',
 'bop': "\n'bop:Butlletí Oficial de la Província'",
 'pcap': '\n\npcap: Project Capture Plan',
 'ceip': '\n\nCEIP: Centro de Educación Infantil y Primaria (Center for Infant and Primary Education)',
 'jjmm': '\n\nJJMM: Jocs del Mediterrani del 2018',
 'smp': '\n\nSMP: Social Media Plan',
 'avda': '\n\nAVDA: Avenida',
 'ffcc': '\n\nffcc: Ferrocarrils de la Generalitat de Catalunya (FFCC)',
 'itv': '\n\nITV: Inspección Técnica Periódica del Vehículo'}